In [ ]:
import numpy as np

#The operations on convergence maps are handled with the ConvergenceMap class
from lenstools import ConvergenceMap

import time
import os

In [ ]:
######################## define constants etc. ############################### 

workPath = '/n/holyscratch01/dvorkin_lab/Users/tianliwang/'
mapPath = workPath+'maps_unzipped/'

n_cosmo = 2
n_imagespercosmo = 512  # number of images per cosmology 

Om_d = 0.311
si_d = 0.789

l_edges = np.logspace(np.log10(100.0), np.log10(3.75e4), 21)

In [ ]:
def read_parametersfromfile(path):
    '''
    Input: string for the directory to the folders of cosmology 
    
    Function: reads the Omega_m and sigma_8 values from the folder names in 
    the path directory and store them as arrays of strings. Stores the values 
    as strings to avoid rounding, for example, 0.600 to 0.6. 
    
    Output: (array of strings of Omega_m, array of strings of sigma_8)
    '''
    
    Om_strings = [] 
    si_strings = []
    
    for filename in os.listdir(path):
        if (filename.startswith('Om') and not filename.endswith('.gz')): 
            Om_strings.append(filename[2:7]) 
            si_strings.append(filename[10:15])
            
    return Om_strings, si_strings


def get_pls(path, Om, si, l_edges, i_start, i_end):
    '''
    Input: path to the maps (string), omegam and sigma8 value for the maps (floats), 
    l edges for power spectrum (array), start index and end index for reading the images (ints)
    
    Function: get the power spectra for all the maps using ConvergenceMap in LensTools with the 
    input l_edges 
    
    Output: l, matrix of power spectra of dimension (i_end+1-i_start)x(l_edges-1) (each row for one map)
    
    '''
    
    Pls = [] 
    
    for i in range(i_start, i_end+1): 
        if (i < 10): filenum = '00{}'.format(i)
        elif (i < 100): filenum = '0{}'.format(i)
        else: filenum = i 
        
        map_filename = '{}Om{}_si{}/WLconv_z1.00_0{}r.fits'.format(path, Om, si, filenum)
        conv_map = ConvergenceMap.load(map_filename)
        
        # Measure the power spectrum calling the powerSpectrum method
        l, Pl = conv_map.powerSpectrum(l_edges)
        
        Pls.append(Pl)
        
    return l, Pls


def get_p(Pls, d): 
    '''
    Input: matrix of power spectra of dimension (# maps)x(n_lbins) and the 
    discriptor array (mean power spec of one chosen cosmo)
    
    Function: calculates the posterior value for 
    '''
    
    N = len(Pls)    # number of maps 
    
    mu = np.mean(Pls, axis=0)
    assert len(mu) == len(Pls[0])   # check if taking mean along correct dimension 
    
    cov = np.cov(np.array(Pls).T)
    covinv = np.linalg.inv(cov)
    covinv = (N-len(mu)-2)/(N-1) * covinv  # debias the cov inverse 
    
    exponent = -0.5*np.dot(d-mu, np.dot(covinv, d-mu))
    
    return np.exp(exponent)

In [ ]:
Oms_all, sis_all = read_parametersfromfile(mapPath)
Oms = Oms_all[:n_cosmo]
sis = sis_all[:n_cosmo]

In [ ]:
# some testing

# get the mean of the cosmo chosen for d 
_, Pls_d = get_pls(mapPath, Om_d, si_d, l_edges, 1, n_imagespercosmo)
mu_d = np.mean(Pls_d, axis=0)

# test on a few cosmos 
for i in range(n_cosmo): 
    l, Pls = get_pls(mapPath, Oms[i], sis[i], l_edges, 1, n_imagespercosmo)
    
    p = get_p(Pls, mu_d)
    print(p)

In [ ]:
# calculate and save spectra, each cosmo in one column, with Om and si in the first two rows
## I guess we don't need to save l here?

spectra = []

for i in range(n_cosmo): 
    l, Pls = get_pls(mapPath, Oms[i], sis[i], l_edges, 1, n_imagespercosmo)
#    Pls = (i+1) * np.ones((n_imagespercosmo, len(l_edges)-1))  # for testing without lenstools; use the line above!
    flatPls = np.ndarray.flatten(Pls)
    spectra = spectra + flatPls.tolist()

spectra = np.reshape(spectra, (n_cosmo, -1)).T
spectraWithLabels = np.concatenate(([np.array(Oms), np.array(sis)], spectra), axis=0)  # append list of Oms and sis to the spectra array

np.save(workPath+'spectra.npy', spectraWithLabels)

In [ ]:
# check dimenstions of arrays
print(spectra.shape)
print(spectraWithLabels.shape)